In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import preprocessing.generate_preprocessed_dataset as pp
from training.temp_attention_training import run_temp_model_training, create_temporal_pairs, create_dataloaders
import time

In [3]:
# X, y, groups, activities = pp.preprocessing(n_epochs=5)

In [4]:
# X.shape, y.shape, groups.shape, activities.shape

In [5]:
start_time = time.time()
loss_dict, best_model = run_temp_model_training(1000)
end_time = time.time()
print(f"Training completed in {(end_time - start_time) / 3600:.2f} hours.")

The device used for training is: cuda
Starting training of split: [ 2 10 15  6]
Starting training of split: [ 5  3  1 12]
Starting training of split: [ 8  4 14 11]
Starting training of split: [ 7  9 13]
Training completed in 2.89 hours.


In [ ]:
X, y, groups, activity = pp.preprocessing(n_epochs=None)
X = X[:, 0, :]
X, y, groups, activity = create_temporal_pairs(X, y, groups, activity)
val_loader = create_dataloaders(X, y, batch_size=1)

In [ ]:
X.shape
for X[]

(64667, 1, 256, 2)

In [12]:
import numpy as np
np.unique(y)

array([ 41.69085059,  41.7219776 ,  41.82354708, ..., 186.80862433,
       186.84242878, 187.01542534])

In [ ]:
best_model()

KID_PPG(
  (conv_block1): ConvolutionBlock(
    (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu1): ReLU()
    (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu2): ReLU()
    (conv3): Conv1d(32, 32, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu3): ReLU()
    (avg_pooling): AvgPool1d(kernel_size=(4,), stride=(4,), padding=(0,))
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (conv_block2): ConvolutionBlock(
    (conv1): Conv1d(32, 48, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu1): ReLU()
    (conv2): Conv1d(48, 48, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu2): ReLU()
    (conv3): Conv1d(48, 48, kernel_size=(5,), stride=(1,), dilation=(2,))
    (relu3): ReLU()
    (avg_pooling): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (conv_block3): ConvolutionBlock(
    (conv1): Conv1d(48, 64, kernel_size=(5,), stride=(1,), dilation=(2,))
   